In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

In [2]:
# Εισαγωγή διαφόρων csv

df_icu = pd.read_csv('D:\Papers\Cyprus_Paper\Data\o46_chart_lab_with_death_flag.csv')

df_temp = pd.read_csv('D:\Papers\Cyprus_Paper\Data\o46_chart_lab_with_death_flag.csv', usecols = ['subject_id', 'hadm_id', 'date', 'time'])

df_temp.to_csv('D:\Papers\Cyprus_Paper\Data\o47_count_patients.csv', index=False)

df_patients = pd.read_csv('D:\MSc Aegean\Thesis\mimic-iv-1.0\core\patients.csv.gz')

df_admissions = pd.read_csv('D:\MSc Aegean\Thesis\mimic-iv-1.0\core\Admissions.csv.gz', usecols = ['subject_id', 'hadm_id', 'ethnicity'])

In [3]:
# Αυτό το φτιάχνω στο excel περνώντας την αρίθμηση με συνάρτηση.
df_count_24hours = pd.read_csv('D:\Papers\Cyprus_Paper\Data\o48_count_24_hours.csv', usecols = ['twenty_four_hours'])

df_rslt=pd.merge(df_icu, df_count_24hours, left_index=True, right_index=True)

# Ενσωμάτωση δημογραφικών

In [4]:
df_rslt['date'] = pd.to_datetime(df_rslt['date'])

df_rslt['year'] = df_rslt['date'].dt.year

df_patients['dod'] = pd.to_datetime(df_patients['dod'])

In [5]:
"""
Δημιουργία ξεχωριστά όλων των 24ώρων αρχείων
με τις καταγραφές των ασθενών που περιλαμβάνουν
ηλικία, φύλλο, εθνικότητα.
"""

for i in range (1, (df_rslt.twenty_four_hours.max()+1)):
    # Επιλογή 24ώρου
    temp = df_rslt.loc[df_rslt['twenty_four_hours'] == i]
    # Στο ενδεχόμενο να υπάρχουν διπλές εγγραφές
    final = temp.drop_duplicates(subset=['subject_id'])
    # ενοποίηση του 24ώρου με φύλλο, χρονιές
    rslt_df=pd.merge(final, df_patients, on=['subject_id'], how='left')
    # ενοποίηση με εθνικότητα
    rslt_df=pd.merge(rslt_df, df_admissions, on=['subject_id', 'hadm_id'], how='left')
    # υπολογισμός ηλικίας
    rslt_df['temp_age'] = rslt_df['year'] - rslt_df['anchor_year']
    # υπολογισμός ηλικίας
    rslt_df['Age'] = rslt_df['temp_age'] + rslt_df['anchor_age']
    # υπολογισμός επιβίωσης
    rslt_df['Survival Days'] = rslt_df['dod'].sub(rslt_df['date'], axis=0)
    # αφαίρεση μη χρήσιμων στηλών
    rslt_df = rslt_df.drop(['year', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod', 'temp_age', 'twenty_four_hours' ], axis=1)
    # εξαγωγή csv
    rslt_df.to_csv('D:\Papers\Cyprus_Paper\Data\Twenty_four_hours\o{}.csv'.format(i), index=False)